In [7]:
using MLJ
using ScikitLearn
using Random, Statistics
using CSV
using DataFrames

In [8]:
# read the CSV file
df = CSV.read("hikari.csv", DataFrame)

df = select(df, [:"idle.tot", :"flow_iat.tot", :"fwd_iat.tot", :"active.avg", :"idle.max", :"flow_iat.max", :"fwd_iat.max", :"active.max", :"idle.avg", :"active.tot", :"bwd_iat.max", :"idle.min", :"flow_iat.avg", :"fwd_iat.avg", :"flow_iat.min", :"fwd_iat.min", :"fwd_iat.std", :"bwd_iat.std", :"flow_iat.std", :"bwd_iat.avg", :"Label"])



train_ratio = 0.8
test_ratio = 1.0 - train_ratio

nrows = nrow(df)
train_indices = randsubseq(1:nrows, train_ratio)
test_indices = setdiff(1:nrows, train_indices)

train_df = df[train_indices, :]
test_df = df[test_indices, :]

Row,idle.tot,flow_iat.tot,fwd_iat.tot,active.avg,idle.max,flow_iat.max,fwd_iat.max,active.max,idle.avg,active.tot,bwd_iat.max,idle.min,flow_iat.avg,fwd_iat.avg,flow_iat.min,fwd_iat.min,fwd_iat.std,bwd_iat.std,flow_iat.std,bwd_iat.avg,Label
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64
1,0.0,4.57145e6,4.57145e6,4.57145e6,0.0,4.34038e6,4.34038e6,4.57145e6,0.0,4.57145e6,4.40903e6,0.0,1.75825e5,3.5165e5,6.91414,20.9808,1.19874e6,1.26934e6,8.49506e5,3.78542e5,1
2,1.15539e7,1.17865e7,1.17865e7,1.16319e5,1.15539e7,1.15539e7,1.15539e7,2.03899e5,1.15539e7,2.32638e5,1.16218e7,1.15539e7,4.53327e5,9.06654e5,10.9673,19.0735,3.1992e6,3.35138e6,2.26411e6,9.79817e5,1
3,9.30198e6,9.57994e6,9.57994e6,1.3898e5,9.30198e6,9.30198e6,9.30198e6,2.49082e5,9.30198e6,2.77961e5,9.37005e6,9.30198e6,3.42141e5,6.84282e5,14.0667,52.9289,2.48048e6,2.59467e6,1.75602e6,7.34629e5,1
4,6.86407e6,7.33164e6,7.33164e6,233783.0,6.86407e6,6.86407e6,6.86407e6,4.38243e5,6.86407e6,467566.0,6.93283e6,6.86407e6,2.71542e5,5.63972e5,5.96046,15.0204,1.89487e6,1.91544e6,1.31839e6,5.6168e5,1
5,0.0,2.43333e6,2.43333e6,2.43333e6,0.0,1.96707e6,1.96707e6,2.43333e6,0.0,2.43333e6,2.0349e6,0.0,93589.5,187179.0,15.0204,40.0543,5.40195e5,5.81771e5,385138.0,2.00328e5,1
6,6.585e6,6.81652e6,6.81652e6,1.1576e5,6.585e6,6.585e6,6.585e6,202466.0,6.585e6,2.31521e5,6.65329e6,6.585e6,2.62174e5,5.24347e5,9.05991,19.0735,1.82118e6,1.91717e6,1.28967e6,5.65599e5,1
7,0.0,4.60684e6,4.60684e6,4.60684e6,0.0,4.36923e6,4.36923e6,4.60684e6,0.0,4.60684e6,4.43806e6,0.0,1.77186e5,3.54373e5,7.86781,17.8814,1.2066e6,1.27757e6,855112.0,3.81422e5,1
8,8.57071e6,8.80569e6,8.80569e6,1.17489e5,8.57071e6,8.57071e6,8.57071e6,2.05386e5,8.57071e6,2.34979e5,8.64027e6,8.57071e6,3.3868e5,6.77361e5,22.8882,35.0475,2.3718e6,2.49068e6,1.67906e6,7.31393e5,1
9,6.37308e6,6.60232e6,6.60232e6,1.14624e5,6.37308e6,6.37308e6,6.37308e6,200243.0,6.37308e6,2.29247e5,6.44144e6,6.37308e6,2.53936e5,5.07871e5,24.0803,36.9549,1.76246e6,1.85607e6,1.24813e6,5.47772e5,1


In [9]:
using ScikitLearn
@sk_import ensemble:AdaBoostClassifier

PyObject <class 'sklearn.ensemble._weight_boosting.AdaBoostClassifier'>

In [10]:
# converting into vectors
Ada_X_train = Matrix(select(train_df, Not(:Label)))

Ada_y_train = Vector(train_df.Label)

Ada_X_test = Matrix(select(test_df, Not(:Label)))

Ada_y_test = Vector(test_df.Label)

111555-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 1
 1
 1
 1
 1
 1
 1
 1
 1

In [11]:
# Create an AdaBoostClassifier
ada_model = AdaBoostClassifier(n_estimators=50, random_state=0)

PyObject AdaBoostClassifier(random_state=0)

In [14]:
# Fit the model to your data
ScikitLearn.fit!(ada_model, Ada_X_train, Ada_y_train)

PyObject AdaBoostClassifier(random_state=0)

In [17]:
# Make predictions
ada_y_pred = ScikitLearn.predict(ada_model, Ada_X_test)


#accuracy
accuracy = mean(ada_y_pred .== Ada_y_test)
# Calculate True Positives (TP), True Negatives (TN), False Positives (FP), and False Negatives (FN)
TP = sum((ada_y_pred .== 1) .& (Ada_y_test .== 1))
TN = sum((ada_y_pred .== 0) .& (Ada_y_test .== 0))
FP = sum((ada_y_pred .== 1) .& (Ada_y_test .== 0))
FN = sum((ada_y_pred .== 0) .& (Ada_y_test .== 1))

# Calculate precision, recall, and F1-score
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)

println("Accuracy: $accuracy")
println("Precision: $precision")
println("Recall: $recall")
println("F1-Score: $f1_score")

Accuracy: 0.9321500605082694
Precision: 0.5070104025327906
Recall: 0.1475
F1-Score: 0.22851900927530322
